In [19]:
import numpy as np
import pandas as pd

In [20]:
train_data = pd.read_csv('train.csv')
train_data

,comment,price_value
0,قیمت مناسب وکیفیت خوب پیشنهادمیکنم حتما خرید کنید,1
1,به اندازه یک میلیمتر دورتادور گوشی خالی میماند...,0
2,از همه نظر عالی و یک خرید خوب در قیمت حدود۴۰ ...,1
3,فقط یک بار هر یک ربع ساعت 1 درصد شارژ کرد بعدش...,0
4,قیمت این کالا خیلی تغییر میکنه . من خریدم چندر...,1
...,...,...
39995,خیلی خوبه واسه گوشی m20ولی یه ترک از پایین داش...,0
39996,چند روزه دارم استفاده میکنم در یک کلام عالیه\r...,1
39997,من سی تومن خریدم و با توجه به قیمت ارزش خرید د...,1
39998,عالیه از هر نظر \nهم قیمتش کمه \nهم قطعاتش زیا...,0


In [21]:
test_data = pd.read_csv('train.csv')
test_data

,comment,price_value
0,قیمت مناسب وکیفیت خوب پیشنهادمیکنم حتما خرید کنید,1
1,به اندازه یک میلیمتر دورتادور گوشی خالی میماند...,0
2,از همه نظر عالی و یک خرید خوب در قیمت حدود۴۰ ...,1
3,فقط یک بار هر یک ربع ساعت 1 درصد شارژ کرد بعدش...,0
4,قیمت این کالا خیلی تغییر میکنه . من خریدم چندر...,1
...,...,...
39995,خیلی خوبه واسه گوشی m20ولی یه ترک از پایین داش...,0
39996,چند روزه دارم استفاده میکنم در یک کلام عالیه\r...,1
39997,من سی تومن خریدم و با توجه به قیمت ارزش خرید د...,1
39998,عالیه از هر نظر \nهم قیمتش کمه \nهم قطعاتش زیا...,0


In [22]:
from hazm import Normalizer, word_tokenize, Stemmer, stopwords_list
import re

normalizer = Normalizer()
stemmer = Stemmer()
stop_words = set(stopwords_list())

def preprocessing(comment):
    filtered = []
    
    # نرمال‌سازی
    comment = normalizer.normalize(comment)

    # حذف اعداد و نمادها (فقط حروف و فاصله نگه داشته می‌شود)
    comment = re.sub(r'[^\u0600-\u06FF\s]', '', comment)  # حذف غیرحروف فارسی

    # توکن‌سازی
    tokens = word_tokenize(comment)

    for token in tokens:
        if token not in stop_words and len(token) > 1:
            stemmed = stemmer.stem(token)
            filtered.append(stemmed)

    return filtered

In [23]:
train_data['tokens'] = train_data['comment'].apply(preprocessing)

In [24]:
prior_probability = train_data['price_value'].value_counts(normalize=True).to_dict()
print(prior_probability)

{0: 0.520025, 1: 0.479975}


In [25]:
from sklearn.model_selection import train_test_split
x = train_data['comment']
y = train_data['price_value']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

df_train = pd.DataFrame({'comment': x_train, 'price_value': y_train})
df_train['price_value'].value_counts(normalize=True)
df_test = pd.DataFrame({'comment': x_test, 'price_value': y_test})